In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import seaborn as sns
import collections
import json

In [ ]:
prov_ter = collections.OrderedDict()
prov_ter = {
    "Federal",
    "BC",
    "MB",
    "NB",
    "NL",
    "NS",
    "ON",
    "PE",
    "QC",
    "SK",
    "NT",
    "YT",
    "FED"}

fake_brackets_prov = [
    (0, 15),
    (40000, 22),
    (90000, 26),
    (140000, 29)
]

fake_brackets_fed = [
    (0, 20),
    (50000, 27),
    (100000, 34),
    (160000, 35)
]
years = xrange(1985, 2016)
incomes = xrange(5000, 2000000, 5000)

In [ ]:
tax_brackets = collections.OrderedDict()
for x in years:
    tax_brackets[str(x)] = {}

In [ ]:
for year in tax_brackets:
    for pt in prov_ter:
        if pt != "FED":
            tax_brackets[year][pt] = fake_brackets_prov
        else:
            tax_brackets[year][pt] = fake_brackets_fed

for year in xrange(1999, 2016):
    tax_brackets[str(year)]["NU"] = fake_brackets_prov

In [ ]:
tb_json = json.dumps(tax_brackets)

In [ ]:
file_object = open('HistoricalRates/historicalTaxRates.json', 'w')
file_object.write(tb_json)
file_object.close()

In [42]:
from pprint import pprint

with open('Data/incomeTax.json') as data_file:    
    data = json.load(data_file)

print data[0]["data"]["New Brunswick"][0]["rate"]
print data[0]["data"]["New Brunswick"][0]["upper"]

9.68
39305


In [63]:
def findBracket(year, govt, income, data):
    fed_brackets = data["Federal"]
    prov_brackets = data[govt]
    
    untaxed_income = income
    fed_tax_paid = 0.0
    lower = 0
    n = 0
    while untaxed_income > 0:
        print n
        bracket = fed_brackets[n]
        try:
            upper = int(bracket["upper"])
            rate = float(bracket["rate"])/100.0
        except:
            upper = "max"
            rate = float(bracket["rate"])/100.0

        if upper == "max" or income < upper:
            fed_tax_paid = fed_tax_paid + untaxed_income*rate
            untaxed_income = 0
        else:
            fed_tax_paid = fed_tax_paid + (upper - lower)*rate
            untaxed_income = untaxed_income - (upper - lower)
            lower = upper
            n = n + 1
    untaxed_income = income
    avg_fed_tax = round(fed_tax_paid / income * 100.0, 2)

    return {"fed": avg_fed_tax}

In [64]:
year = data[0]["year"]
year_data = data[0]["data"]
tax = findBracket(year, "Nova Scotia", 1000000, year_data)
print tax

0
1
2
3
{'fed': 27.93}


In [ ]:
def allBracketsForYear(year, province, incomeList, brackets):
    incomeTotals = collections.OrderedDict()
    for income in incomeList:
        taxes = findBracket(year, province, income, brackets)
        incomeTotals[income] = taxes["total"]
    return incomeTotals
totals = allBracketsForYear("2000", "NB", incomes, tax_brackets)

In [ ]:
total_tax_year_provt = collections.OrderedDict()
for year in tax_brackets:
    for provt in tax_brackets[year]:
        if provt != "FED":
            # total_tax_year_provt[year][provt] = 